<a href="https://colab.research.google.com/github/TAUforPython/machinelearning/blob/main/example_GNN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
export TORCH=2.5.1
export CUDA=cu121

In [2]:
!pip3 install -q --pre torch -f https://download.pytorch.org/whl/sparse/cpu/torch_sparse.html
!pip3 install -q --no-index torch-sparse -f https://pytorch-geometric.com/whl/torch-2.5.1+cu121.html
!pip3 install -q pyg-lib -f https://pytorch-geometric.com/whl/torch-2.5.1+cu121.html

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.3 MB/s eta 0:00:00


In [3]:
!pip3 install -q ogb
!pip3 install -q torch_geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.2 MB/s eta 0:00:00


In [4]:
import os
import torch
import torch.nn.functional as F
from tqdm import tqdm
from torch_geometric.loader import NeighborLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.nn import MessagePassing, SAGEConv
from ogb.nodeproppred import Evaluator, PygNodePropPredDataset


import torch_geometric.transforms as T

Graph: The ogbn-arxiv dataset is a directed graph, representing the citation network between all Computer Science (CS) arXiv papers indexed by MAG [1]. Each node is an arXiv paper and each directed edge indicates that one paper cites another one. Each paper comes with a 128-dimensional feature vector obtained by averaging the embeddings of words in its title and abstract. The embeddings of individual words are computed by running the skip-gram model [2] over the MAG corpus. We also provide the mapping from MAG paper IDs into the raw texts of titles and abstracts here. In addition, all papers are also associated with the year that the corresponding paper was published.

Graph: The ogbn-proteins dataset is an undirected, weighted, and typed (according to species) graph. Nodes represent proteins, and edges indicate different types of biologically meaningful associations between proteins, e.g., physical interactions, co-expression or homology [1,2]. All edges come with 8-dimensional features, where each dimension represents the approximate confidence of a single association type and takes values between 0 and 1 (the larger the value is, the more confident we are about the association). The proteins come from 8 species.

In [6]:
target_dataset = 'ogbn-arxiv'

# Это загрузит ogbn-arxiv в папку "networks"
dataset = PygNodePropPredDataset(name=target_dataset, root='networks')


#target_dataset = 'ogbn-proteins'
#dataset = PygNodePropPredDataset(name='ogbn-proteins', transform=T.ToSparseTensor(attr='edge_attr'))
#dataset = PygNodePropPredDataset(name=target_dataset, root='networks')

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:03<00:00, 24.62it/s]


Extracting networks/arxiv.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 2286.97it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 458.80it/s]

Saving...



Done!
/usr/local/lib/python3.11/dist-packages/ogb/nodeproppred/dataset_pyg.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(self.pro

In [7]:
data = dataset[0]
#data

data.x: Node feature matrix with shape [num_nodes, num_node_features]

data.edge_index: Graph connectivity in COO format with shape [2, num_edges] and type torch.long

data.edge_attr: Edge feature matrix with shape [num_edges, num_edge_features]

data.y: Target to train against (may have arbitrary shape), e.g., node-level
targets of shape [num_nodes, *] or graph-level targets of shape [1, *]

data.pos: Node position matrix with shape [num_nodes, num_dimensions]


In [8]:
split_idx = dataset.get_idx_split()
train_idx = split_idx['train']
valid_idx = split_idx['valid']
test_idx = split_idx['test']

In [9]:
train_loader = NeighborLoader(data, input_nodes=train_idx,
                               shuffle=True, num_workers=os.cpu_count() - 2,
                               batch_size=1024, num_neighbors=[30] * 2)
total_loader = NeighborLoader(data, input_nodes=None, num_neighbors=[-1],
                               batch_size=4096, shuffle=False,
                               num_workers=os.cpu_count() - 2)

In [10]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels,
                 hidden_channels, out_channels,
                 n_layers=2):
      super(SAGE, self).__init__()
      self.n_layers = n_layers
      self.layers = torch.nn.ModuleList()
      self.layers_bn = torch.nn.ModuleList()
      if n_layers == 1:
        self.layers.append(SAGEConv(in_channels, out_channels,   normalize=False))
      elif n_layers == 2:
        self.layers.append(SAGEConv(in_channels, hidden_channels, normalize=False))
        self.layers_bn.append(torch.nn.BatchNorm1d(hidden_channels))
        self.layers.append(SAGEConv(hidden_channels, out_channels, normalize=False))
      else:
        self.layers.append(SAGEConv(in_channels, hidden_channels, normalize=False))
        self.layers_bn.append(torch.nn.BatchNorm1d(hidden_channels))
      for _ in range(n_layers - 2):
        self.layers.append(SAGEConv(hidden_channels,  hidden_channels, normalize=False))
        self.layers_bn.append(torch.nn.BatchNorm1d(hidden_channels))
        self.layers.append(SAGEConv(hidden_channels, out_channels, normalize=False))
      for layer in self.layers:
        layer.reset_parameters()


    def forward(self, x, edge_index):
      if len(self.layers) > 1:
        looper = self.layers[:-1]
      else:
        looper = self.layers
      for i, layer in enumerate(looper):
        x = layer(x, edge_index)
        try:
          x = self.layers_bn[i](x)
        except Exception as e:
          abs(1)
        finally:
          x = F.relu(x)
          x = F.dropout(x, p=0.5, training=self.training)

      if len(self.layers) > 1:
        x = self.layers[-1](x, edge_index)

      return F.log_softmax(x, dim=-1), torch.var(x)

    def inference(self, total_loader, device):
      xs = []
      var_ = []
      for batch in total_loader:
        out, var = self.forward(batch.x.to(device), batch.edge_index.to(device))
        out = out[:batch.batch_size]
        xs.append(out.cpu())
        var_.append(var.item())

      out_all = torch.cat(xs, dim=0)
      return out_all, var_

In [11]:
print(data.num_features)


128


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SAGE(data.num_features, 256, dataset.num_classes, n_layers=2)

#model = SAGE(data.x.shape[1], 256, dataset.num_classes, n_layers=2)
model.to(device)
epochs = 2
optimizer = torch.optim.Adam(model.parameters(), lr=0.03)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=7)

In [13]:
def test(model, device):
  evaluator = Evaluator(name=target_dataset)
  model.eval()
  out, var = model.inference(total_loader, device)

  y_true = data.y.cpu()
  y_pred = out.argmax(dim=-1, keepdim=True)
  train_acc = evaluator.eval({
      'y_true': y_true[split_idx['train']],
      'y_pred': y_pred[split_idx['train']],
      })['acc']
  val_acc = evaluator.eval({
      'y_true': y_true[split_idx['valid']],
      'y_pred': y_pred[split_idx['valid']],
      })['acc']
  test_acc = evaluator.eval({
      'y_true': y_true[split_idx['test']],
      'y_pred': y_pred[split_idx['test']],
      })['acc']
  return train_acc, val_acc, test_acc, torch.mean(torch.Tensor(var))

Prediction task: The task is to predict the 40 subject areas of arXiv CS papers, e.g., cs.AI, cs.LG, and cs.OS, which are manually determined (i.e., labeled) by the paper’s authors and arXiv moderators. With the volume of scientific publications doubling every 12 years over the past century, it is practically important to automatically classify each publication’s areas and topics. Formally, the task is to predict the primary categories of the arXiv papers, which is formulated as a 40-class classification problem.

Prediction task: The task is to predict the presence of protein functions in a multi-label binary classification setup, where there are 112 kinds of labels to predict in total. The performance is measured by the average of ROC-AUC scores across the 112 tasks.

In [14]:
print(data.y)

tensor([[ 4],
        [ 5],
        [28],
        ...,
        [10],
        [ 4],
        [ 1]])


In [15]:
for epoch in range(1, epochs):
  model.train()

  pbar = tqdm(total=train_idx.size(0))
  pbar.set_description(f'Epoch {epoch:02d}')

  total_loss = total_correct = 0

  for batch in train_loader:
    batch_size = batch.batch_size
    optimizer.zero_grad()

    #out, _ = model(batch.x.to(device), batch.edge_index.to(device))
    out, _ = model(batch.x, batch.edge_index)
    out = out[:batch_size]

    batch_y = batch.y[:batch_size].to(device)
    batch_y = torch.reshape(batch_y, (-1,))

    loss = F.nll_loss(out, batch_y)
    loss.backward()
    optimizer.step()

    total_loss += float(loss)
    total_correct += int(out.argmax(dim=-1).eq(batch_y).sum())
    pbar.update(batch.batch_size)
  pbar.close()

  loss = total_loss / len(train_loader)
  approx_acc = total_correct / train_idx.size(0)

  train_acc, val_acc, test_acc, var = test(model, device)

  print(f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}, Var: {var:.4f}')

Epoch 01: 100%|██████████| 90941/90941 [00:26<00:00, 3423.74it/s]


Train: 0.5690, Val: 0.5623, Test: 0.5080, Var: 8.4021


In [16]:
print(f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}, Var: {var:.4f}')

Train: 0.5690, Val: 0.5623, Test: 0.5080, Var: 8.4021
